In [ ]:
import os
import json

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
matplotlib.rcParams["animation.embed_limit"] = 256
matplotlib.rcParams["font.size"] = 22

import IPython

import daisy
import daisy.daisy_world_rl
from daisy.daisy_world_rl import RLDaisyWorld
from daisy.agents.greedy import Greedy


from daisy.daisy_world_simple import SimpleDaisyWorld

#from daisy.agents.greedy import Greedy

# functions for animating daisyworld env
from daisy.notebook_helpers import tensor_to_image,\
        plot_grid,\
        get_update_fig,\
        seed_all
        
my_seed = 42 

In [ ]:
# DaisyWorld (Based on Watson and Lovelock 1983)

daisy_world = SimpleDaisyWorld()

daisy_world.min_L = 0.55
daisy_world.initial_L = daisy_world.min_L
daisy_world.max_L = 2.0
daisy_world.steps_per_period = 10000
daisy_world.reset()
daisy_world.run_sim()
fig, ax = daisy_world.plot_curve(show_habitable=True)

fig.suptitle("Simple DaisyWorld (Watson & Lovelock 1983)", fontsize=24)
plt.savefig("simple_daisy_world.png")
plt.show()

In [ ]:
def run_q2_sims(q2s, env):
    
    all_lifeless_temps = []
    all_temps = []
    all_dark_daisies = []
    all_light_daisies = []
    all_luminosities = []
    
    for ii, q2 in enumerate(q2s):
    
        if q2 == 0:
            env.q2 = 0
        else:
            env.q2 = env.q / q2
            
        env.reset()
        
        lifeless_temp = []    
        my_temps = []
        my_dark_daisies = []
        my_light_daisies = []
        my_luminosities = []
        
        for step in range(env.ramp_period):
            env.step()
            
            my_temps.append(env.temp.mean())
            my_light_daisies.append(env.grid[:,1,:,:].mean())
            my_dark_daisies.append(env.grid[:,2,:,:].mean())
            my_luminosities.append(env.L)
            
            lifeless_temp.append(env.dead_temp.mean())
                
        all_temps.append(my_temps)
        all_light_daisies.append(my_light_daisies)
        all_dark_daisies.append(my_dark_daisies)
        all_lifeless_temps.append(lifeless_temp)
        all_luminosities.append(my_luminosities)

    return all_lifeless_temps, all_temps,\
            all_light_daisies, all_dark_daisies,\
            all_luminosities
        
def plot_daisyworld_trajectories(q2_values, lifeless_temp, temps,\
        dark_daisies, light_daisies, luminosities, temp_optimal=295.5):
    
    my_cmap = plt.get_cmap("magma")
    fig, ax = plt.subplots(2, 1, figsize=(10,10), facecolor="white")
    ax2 = ax[1].twinx()
    
    
    lines = []
    
    # dead temp  
    dead_temp = lifeless_temp[0]
    # habitable range
    t_range = np.sqrt(1 / 0.003265)
    x = [elem for elem in range(len(dead_temp))]
    lower = [temp_optimal - t_range, temp_optimal - t_range]
    upper = [temp_optimal + t_range, temp_optimal + t_range]
    
    habitable_x = [x[0], x[-1]]
    lines += ax2.plot(habitable_x, [temp_optimal, temp_optimal], alpha=0.3, label="Habitable Range")
    ax2.fill_between(habitable_x, lower, upper, alpha=0.05)
    
    lines += ax[1].plot(x, luminosities[0], "--", lw=4, color=[0.6,0.6,0.6], alpha=0.33, label="Stellar Luminosity")
    lines += ax2.plot(x, dead_temp, "-.", lw=4, color=[0,0,0], alpha=0.33, label="Lifeless Temp.")
    
    for kk, q2 in enumerate(q2_values):
        my_color = list(my_cmap(kk / (len(q2_values)*2) )) 
        
        if q2 == 0:
            ax[0].plot(x, dark_daisies[kk], ":",lw=6, color=my_color,\
                       alpha=0.25, label=f"Dark Daisies $q_2$ = {q2}")
            ax[0].plot(x, light_daisies[kk], "-",lw=6, color=my_color,\
                       alpha=0.25, label=f"Light Daisies $q_2$ = {q2}")

            lines += ax2.plot(x, temps[kk], lw=6, color=my_color,\
                       alpha=0.5, label=f"$q_2$ = {q2}")
        else:
            ax[0].plot(x, dark_daisies[kk], ":",lw=6, color=my_color,\
                       alpha=0.25, label=f"Dark Daisies $q_2$ =q/{q2}")
            ax[0].plot(x, light_daisies[kk], "-",lw=6, color=my_color,\
                       alpha=0.25, label=f"Light Daisies $q_2$ =q/{q2}")

            lines += ax2.plot(x, temps[kk], lw=6, alpha=0.5, label=f"$q_2$ = $q$/{q2}")
    
    ax[0].legend
    ax[0].set_xticklabels("")
    ax2.set_ylabel("Temperature (K)")
    ax[0].set_ylabel("Daisy Coverage")
    ax[1].set_ylabel("Stellar Luminosity")
    
    labels = [line.get_label() for line in lines]
    
    ax[1].axis([0, max(x), 0.65, 1.5])
    
    ax2.axis([0, max(x), 273, 340])
    ax[0].legend(loc=[1.15,0.15], facecolor="white")
    ax[1].legend(lines, labels, loc=[1.15,0.15])
    ax[1].set_xlabel("Simulation Step")
    
    fig.suptitle("2D DaisyWorld with Microclimates")
    return fig, ax

In [ ]:
env = RLDaisyWorld()
env.min_L = 0.725
q2s = [0, 64, 8.]

lifeless, temps, light, dark, luminosities = run_q2_sims(q2s, env)

In [ ]:
fig, ax = plot_daisyworld_trajectories(q2s, lifeless, temps,\
        dark, light, luminosities)

plt.savefig("daisyworld_2d.png")

In [ ]:
from daisy.agents.mlp import MLP
from daisy.agents.greedy import Greedy

In [ ]:
directory_filepath = "../results"

dir_list = os.listdir(directory_filepath)

best_fitness = - float("Inf")

for subdir in dir_list:
    subdir_list = os.listdir(os.path.join(directory_filepath, subdir))
    for file in subdir_list:
        if file.endswith("progress.json"):
            
            my_path = os.path.join(directory_filepath, subdir, file)
            
            with open(my_path, "r") as f:
                progress = json.load(f)

            if np.max(progress["max_fitness"]) > best_fitness:
                best_fitness = np.max(progress["max_fitness"])
                print(f"new best fitness in {file}")
            

In [ ]:
best_agent = "cmaes_exp_002_seed11_best_agent_gen127.json"

with open(os.path.join(directory_filepath, best_agent), "r") as f:
    
    my_agent = json.load(f)

In [ ]:
in_dim = my_agent["in_dim"]
out_dim = my_agent["out_dim"]
h_dim = my_agent["h_dim"]

params = np.array(my_agent["parameters"])

kwargs = my_agent
agent = MLP(**kwargs)

agent.set_parameters(params)

In [ ]:
env = RLDaisyWorld()
env.batch_size = 20
env.agent_gamma = 0.0
env.dt = 1.0

seed_all(my_seed)

steps = []

for ii in range(1):
    
    all_done = False
    obs = env.reset()
    done_at = np.zeros((*obs.shape[:1]), dtype=int)
    while not all_done:
        
        obs, reward, done, info = env.step()
        
        
        grid_done = env.grid[:,1:3,:,:].max(axis=(1,2,3), keepdims = False) <= 0.005
        done_at += (1 - 1 * grid_done)
        
        if grid_done.mean() == 1.0:
            all_done = True
            
no_agent_done_at = 1.0 * done_at

In [ ]:
no_agent_done_at.mean(), no_agent_done_at.min(), no_agent_done_at.shape

In [ ]:


for nn in np.arange(1,301,30):
    env = RLDaisyWorld()
    env.batch_size = 10
    env.n_agents = nn

    seed_all(my_seed)

    steps = []
    for ii in range(1):

        all_done = False
        obs = env.reset()
        done_at = np.zeros((*obs.shape[:1]), dtype=int)

        
        while not all_done:

            action = agent(obs)

            obs, reward, done, info = env.step(action)


            grid_done = env.grid[:,1:3,:,:].max(axis=(1,2,3), keepdims = False) <= 0.005
            done_at += (1 - 1 * grid_done)

            if grid_done.mean() == 1.0:
                all_done = True
        

    agent_done_at = 1.0 * done_at
    
    print(action.min(), action.max(), action.mean(),\
          env.n_agents, agent_done_at.mean(), agent_done_at.min(), agent_done_at)

In [ ]:
agent_done_at.mean(), agent_done_at.min(), #agent_done_at

In [ ]:
from importlib import reload
import daisy.agents.greedy
reload(daisy.agents.greedy)

greedy_agent = daisy.agents.greedy.Greedy()

greedy_agent.epsilon = 1.0

for nn in np.arange(1,301,30):
    
    env = RLDaisyWorld()
    env.dt = 1.0
    env.batch_size = 10
    env.n_agents = nn

    seed_all(my_seed)

    steps = []
    for ii in range(1):

        all_done = False
        obs = env.reset()
        done_at = np.zeros((*obs.shape[:1]), dtype=int)

        while not all_done:

            action = greedy_agent(obs)

            obs, reward, done, info = env.step(action)


            grid_done = env.grid[:,1:3,:,:].max(axis=(1,2,3), keepdims = False) <= 0.005
            done_at += (1 - 1 * grid_done)

            if grid_done.mean() == 1.0:
                all_done = True


    greedy_done_at = 1.0 * done_at

    print(env.n_agents, greedy_done_at.mean(), greedy_done_at.min(), greedy_done_at)

In [ ]:
from importlib import reload
import daisy.agents.greedy
reload(daisy.agents.greedy)

greedy_agent = daisy.agents.greedy.Greedy()
#greedy_agent.epsilon = 1.0

for nn in np.arange(1,301,30):
    
    env = RLDaisyWorld()
    env.batch_size = 10
    env.n_agents = nn

    seed_all(my_seed)

    steps = []
    for ii in range(1):

        all_done = False
        obs = env.reset()
        done_at = np.zeros((*obs.shape[:1]), dtype=int)

        while not all_done:

            action = greedy_agent(obs)

            obs, reward, done, info = env.step(action)


            grid_done = env.grid[:,1:3,:,:].max(axis=(1,2,3), keepdims = False) <= 0.005
            done_at += (1 - 1 * grid_done)

            if grid_done.mean() == 1.0:
                all_done = True


    greedy_done_at = 1.0 * done_at

    print(env.n_agents, greedy_done_at.mean(), greedy_done_at.min(), greedy_done_at)

In [ ]:
from importlib import reload
import daisy.agents.greedy
reload(daisy.agents.greedy)

greedy_agent = daisy.agents.greedy.Greedy()
greedy_agent.greedy = False

for nn in np.arange(1,301,30):
    
    env = RLDaisyWorld()
    env.batch_size = 10
    env.n_agents = nn

    seed_all(my_seed)

    steps = []
    for ii in range(1):

        all_done = False
        obs = env.reset()
        done_at = np.zeros((*obs.shape[:1]), dtype=int)

        while not all_done:

            action = greedy_agent(obs)

            obs, reward, done, info = env.step(action)


            grid_done = env.grid[:,1:3,:,:].max(axis=(1,2,3), keepdims = False) <= 0.005
            done_at += (1 - 1 * grid_done)

            if grid_done.mean() == 1.0:
                all_done = True


    greedy_done_at = 1.0 * done_at

    print(env.n_agents, greedy_done_at.mean(), greedy_done_at.min(), greedy_done_at)

In [ ]:
plt.plot(progress["max_fitness"], alpha=0.1, lw=3, label="max fitness")
plt.plot(progress["min_fitness"], alpha=0.1, lw=3, label="min fitness")

plt.plot(progress["mean_fitness"], alpha=0.7, label="mean fitness")

upper = [el1 + np.sqrt(el2) for el1, el2 in zip(progress["mean_fitness"], progress["variance_fitness"])]
lower = [el1 - np.sqrt(el2) for el1, el2 in zip(progress["mean_fitness"], progress["variance_fitness"])]

x = [ii for ii in range(len(upper))]
plt.fill_between(x, lower, upper, label="std. dev. fitness", color="r", alpha=.20)
plt.legend(loc=[1.1,0.1])

In [ ]:
help(plt.fill_between)